# Upptäck signaturer som inte längre används i koden men fortfarande finns med i dokumentationen

Givet genererade signaturer för funktioner i nya och gamla versionen av koden (funktionsnamn, variabelnamn, ev. m.m.) och en dokumentation av koden:
- Sök igenom dokumentationen efter gamla signaturer som har ändrats
- Om dessa förekommer i dokumentationen finns det en chans att man har glömt att uppdatera dem.

Edge cases:
- Funktioner med samma namn i olika delar av koden
- Funktioner med vanliga namn som även används som vanlig text i dokumentationen utan att syfta på någon funktion
- Namnet på en parameter används bara i vissa fall i python, alltså om den byter namn är det inte sagt att anroparna påverkas (skulle kunna ignoreras till en början)
- Aliases av funktionsnamn ex. path i Django

Ännu mer konkret skulle kunna vara att specifiera hur en signatur kan ändras
- Funktionsnamnet kan ändras
- Funktionen kan tas bort
- En parameter kan läggas till / byta namn / tas bort
- En parameter kan få ett nytt default värde
- En parameter kan annoteras med en viss typ

Det allra enklaste fallet att börja titta på är när funktionsnamnet ändras...

In [2]:
import inspect
import re

In [46]:

# Extraherar signaturer (funktionsnamn) vid definitioner
def extract_signatures(string):
  pattern = r"def +([a-zA-Z0-9_]+)"
  matches = re.findall(pattern, string, flags=0)
  return set(matches)

# Extraherar signaturer (funktionsnamn) vid funktionsanrop
def extract_signatures_usage(string):
  pattern = r"\.([a-zA-Z0-9_]+)\("
  matches = re.findall(pattern, string, flags=0)
  return set(matches)

# Extraherar signaturer från en fil, mha någon av ovanstående funktioner
def extract_signatures_file(filename, extract_fn=extract_signatures):
  with open(filename) as f:
      return extract_fn(f.read())

# Upptäck signaturer som har ändrats givet dokumentation, gammal kod och ny kod
# Signaturer som finns med i dokumentationen och i den gamla koden, men inte i den nya
# har sannolikt tagits bort / bytt namn - Flagga dessa!
def check(docs, old_signatures, new_signatures):
  # Find signatures that no longer exist in the code
  to_check = old_signatures.difference(new_signatures)
  for signature in to_check:
    if signature in docs:
      print(f"Doc alert - {signature} is no longer found in the code, but still present in the documentation")

# Testa hur det fungerar på Cowait

In [48]:
!git clone https://github.com/backtick-se/cowait

fatal: destination path 'cowait' already exists and is not an empty directory.


In [49]:
!ls cowait

Dockerfile     build.sh       examples       poetry.lock    setup.py
LICENSE        cloud          images         publish.sh     test
README.md      cowait         k8setup.yml    pyproject.toml test.sh
assets         docs           notebook       pytest.ini


In [65]:
from os import listdir
from os.path import isfile, isdir
import numpy as np
from tqdm import tqdm

class Extractor:

    def __init__(self, fileType: str):
        self.fileType = fileType

    def find_files(self, start: str):
        files = []

        for fd in listdir(start):
            path = f'{start}/{fd}'

            if isfile(path):
                ext = f'.{self.fileType}'
                if fd[-len(ext):] == ext: files.append(path)
            else:
                f = self.find_files(path)
                files += f if f else []

        return files

    def get_content(self, file: str, skipError: bool=True):
        if not isfile(file):
            raise ValueError(f'{file} is not a file')

        try:
            with open(file) as f:
                return f.read()
        except Exception as e:
            if skipError: print(f'Parsing error, skipping file: {file}, {e}')
            else: raise e

    def extract(self, dir: str):
        if not isdir(dir):
            raise ValueError(f'{dir} is not a directory')

        files = self.find_files(dir)
        return np.array([self.get_content(f) for f in tqdm(files)])

In [66]:
signatures_old_code = set()
for file in Extractor('py').find_files('cowait'):
    signatures_old_code = signatures_old_code.union(extract_signatures_file(file))
print(len(signatures_old_code))

454


In [67]:
signatures_doc = set()
for file in Extractor('md').find_files('cowait'):
    signatures_doc = signatures_doc.union(extract_signatures_file(file))
print(len(signatures_doc))

23


In [71]:
# De här signaturerna finns med både i koden och dokumentationen
signatures_doc.intersection(signatures_old_code)

{'add',
 'after',
 'before',
 'deserialize',
 'init',
 'run',
 'serialize',
 'set_called',
 'test_sleep',
 'validate'}

# Testa t.ex att byta namn på `test_sleep` funktionen i koden innan vi kör vidare....

In [69]:
signatures_new_code = set()
for file in Extractor('py').find_files('cowait'):
    signatures_new_code = signatures_new_code.union(extract_signatures_file(file))
print(len(signatures_new_code))

454


In [70]:
check(signatures_doc, signatures_old_code, signatures_new_code)

Doc alert - test_sleep is no longer found in the code, but still present in the documentation
